In [ ]:
!pip -q install tqdm
!pip -q install pandas
!pip -q install sentence-transformers==2.2.2
!pip -q install llama-index==0.9.29
!pip -q install loguru==0.7.0
!pip -q install rich
!pip -q install tiktoken
!pip -q install chromadb

In [ ]:
%load_ext autoreload
%autoreload 2

#standard libraries
import sys
sys.path.append('../')

import json
import os
import time
from typing import List, Tuple
from math import ceil

#external libraries
import pandas as pd
import numpy as np
from rich import print
from rich.pretty import pprint #nifty library for pretty printing
from torch import cuda
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import json
import chromadb
#change path accordingly for local and colab
path = '/content/gdrive/MyDrive/Data_Medical_Rag/ChromaDb'
client = chromadb.PersistentClient(path)
client.heartbeat()



1719757600069970271

In [ ]:
collection = client.get_collection("Doctor_Copilot_chromaDB_Collection")


In [ ]:

collection.query(
    query_texts=["Prevention of diabetes in older adults"],
    n_results=2,
)


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 64.4MiB/s]


{'ids': [['d_2_cont_2', 'd_11_cont_1']],
 'distances': [[0.8127467058934484, 0.9373169392363202]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [['• Symptoms suggestive of diabetes or changes in risk (B) Diet and Physical Activity Recommendations for Adults at Risk for Type 2 Diabetes (i) Follow a Healthy Eating Pattern • Emphasize whole grains, legumes, nuts, fruits, and vegetables and minimize refined and processed foods • A variety of healthy eating patterns include: » Mediterranean-style » Low-carbohydrate » Vegetarian or plant-based » DASH (Dietary Approaches to Stop Hypertension) (ii) Get Regular Physical Activity • ≥150 min/week of moderate-intensity physical activity, such as brisk walking • May include resistance or strength training • Break up prolonged sedentary time (iii) Where to Refer: • Refer adults with overweight or obesity who are at high risk for type 2 diabetes to a recognized diabetes prevention lifestyle change program (cdc.gov/diabetes/prevent

In [ ]:
result = collection.query(
    query_texts=["What are the specific recommendations for screening and treatment of nanoholic fatty liver disease (NAFLD) in people with type 2 diabetes who are over 50 years old"],
    n_results=3,
)
print(result['documents'])

[
    [
        'Consider antiresorptive and osteo-anabolic agents for those with aT-score ≤−2.0 or previous fragility 
fractures. Question: Are people with diabetes at increased risk for cancer? Ans: Diabetes is associated with 
increased risk of cancers of the liver, pancreas, endometrium, colon/rectum, breast, and bladder. Nevertheless, 
cancer screening recommendations are the same for people with diabetes as for those without diabetes. Question: How
prevalent is nonalcoholic fatty liver disease (NAFLD)? Who should be screened for it and how? Ans: i) Prevalence 
Approximately 70% of people with type 2 diabetes have NAFLD.',
        'Indeterminate Risk: Further Testing: Proceed with vibration-controlled transient elastography or an ELF 
(Enhanced Liver Fibrosis) blood test to refine the risk assessment. After further testing: Low Risk: Continue to 
monitor with a repeat of the test in 2-3 years. High Risk: Refer to a gastroenterologist or hepatologist for 
further evaluation and management. High Risk: Immediate Referral: Refer directly to a gastroenterologist or 
hepatologist for advanced management and intervention. This flowchart provides a guideline on how to proceed with 
fibrosis risk assessment in patients using initial noninvasive testing and determining the need for further 
evaluation or routine monitoring based on risk levels. Management Weight loss with intensive lifestyle therapy 
and/or metabolic surgery, as appropriate, is recommended. Pioglitazone and glucagon-like peptide 1 receptor 
agonists are the preferred agents for treatment of hyperglycemia in adults with type 2 diabetes and NASH, unless 
decompensated cirrhosis is present. People with type 2 diabetes and decompensated cirrhosis from NASH should be 
treated with insulin. Statin therapy is safe in the setting of NAFLD.',
        'ii) Screening Goals To identify individuals at risk for complications from nonalcoholic steatohepatitis 
(NASH), such as cirrhosis and hepatocellular carcinoma To prevent death from liver disease iii) High-Risk 
individuals People with central obesity and cardiometabolic risks or insulin resistance Individuals >50 years of 
age Those with persistent high plasma aminotransferase levels (AST/ALT >30 units/L for >6 months) iv) Screening 
Tool (C) Proposed Algorithm for Risk Stratification in Individuals with NAFLD or NASH. Noninvasive Testing for 
Fibrosis (FIB-4) Initial Step: Perform FIB-4 test to assess the risk of fibrosis. Risk Categories: Low Risk: 
Individuals categorized under low risk following the FIB-4 test. Indeterminate Risk: Individuals whose results do 
not clearly categorize them as either low or high risk. High Risk: Individuals identified as high risk for fibrosis
based on the FIB-4 results. Subsequent Actions: Low Risk: Follow-up: Repeat the FIB-4 test in 2-3 years to monitor 
any changes.'
    ]
]